In [15]:
import openai
import json


with open("../API_KEY", "r") as f:
    key = f.read()

openai.api_key = key

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [16]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "What's the weather like in Boston?"},
        {"role": "assistant", "content": None,
                              "function_call": {
                                "arguments": "{\n\"location\": \"Boston, MA\"\n}",
                                "name": "general_context"
                              },
                              "role": "assistant"
                            },
        {"role": "function", "name": 'general_context', "content": "Boston: sunny with scattered clouds, 35.4C, 54% humidity, sunrise at 6:00am, sunset at 7:00pm"},
    ],
)
response

<OpenAIObject chat.completion id=chatcmpl-7TulRYxkL7AUqB6jEBJzsyhgOimLU at 0x1dc590b4770> JSON: {
  "id": "chatcmpl-7TulRYxkL7AUqB6jEBJzsyhgOimLU",
  "object": "chat.completion",
  "created": 1687364069,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny with scattered clouds. The temperature is 35.4 degrees Celsius, and the humidity is at 54%. The sun rose at 6:00 am and will set at 7:00 pm."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 48,
    "total_tokens": 120
  }
}

In [26]:
type(response.)

int

In [9]:
# Step 1, send model the user query and what functions it has access to
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "What's the weather like in Boston?"}
    ],
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ],
    function_call="auto",
)
   
response

<OpenAIObject chat.completion id=chatcmpl-7RAPm31JcZyy444StKJlQI6xbx7xG at 0x1b0ff775c70> JSON: {
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n\"location\": \"Boston, MA\"\n}",
          "name": "get_current_weather"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1686709246,
  "id": "chatcmpl-7RAPm31JcZyy444StKJlQI6xbx7xG",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 82,
    "total_tokens": 99
  }
}

In [10]:
message = response["choices"][0]["message"]
message

<OpenAIObject at 0x1b0ff7acd60> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n\"location\": \"Boston, MA\"\n}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}

In [11]:
# Step 2, check if the model wants to call a function
if message.get("function_call"):
    function_name = message["function_call"]["name"]

    # Step 3, call the function
    # Note: the JSON response from the model may not be valid JSON
    function_response = get_current_weather(
        location=message.get("location"),
        unit=message.get("unit"),
    )

    # Step 4, send model the info on the function call and function response
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": "What is the weather like in boston?"},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )
else:
    second_response = None

second_response

<OpenAIObject chat.completion id=chatcmpl-7RAPnlXTG59ePzRgpBrp6QTyu0czW at 0x1b0ff7c70e0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686709247,
  "id": "chatcmpl-7RAPnlXTG59ePzRgpBrp6QTyu0czW",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 69,
    "total_tokens": 86
  }
}